<a href="https://colab.research.google.com/github/Amrusha99/NLP_J007/blob/master/Word2Vec_similarity_scores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import gensim
import re

In [8]:
df=pd.read_json("http://snap.stanford.edu/data/amazon/productGraph/categoryFiles/reviews_Office_Products_5.json.gz", lines=True)
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A32T2H8150OJLU,B00000JBLH,ARH,"[3, 4]","I bought my first HP12C in about 1984 or so, a...",5,"A solid performer, and long time friend",1094169600,"09 3, 2004"
1,A3MAFS04ZABRGO,B00000JBLH,"Let it Be ""Alan""","[7, 9]",WHY THIS BELATED REVIEW? I feel very obliged t...,5,"Price of GOLD is up, so don't bury the golden ...",1197676800,"12 15, 2007"
2,A1F1A0QQP2XVH5,B00000JBLH,Mark B,"[3, 3]",I have an HP 48GX that has been kicking for mo...,2,"Good functionality, but not durable like old HPs",1293840000,"01 1, 2011"
3,A49R5DBXXQDE5,B00000JBLH,R. D Johnson,"[7, 8]",I've started doing more finance stuff recently...,5,One of the last of an almost extinct species,1145404800,"04 19, 2006"
4,A2XRMQA6PJ5ZJ8,B00000JBLH,Roger J. Buffington,"[0, 0]",For simple calculations and discounted cash fl...,5,Still the best,1375574400,"08 4, 2013"


In [19]:
import nltk
from nltk.corpus import stopwords
set(stopwords.words('english'))

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'only',
 'or',
 'other',
 'our',
 'ours',
 'ourselves',
 'out',
 'over',
 'own',
 'r

In [0]:
stop_words = set(stopwords.words('english'))

In [0]:
#Removing punctuations and stop words
def rem_punct_stopwords(str):
  words=list(re.findall(r"[\w']+", str.lower()))
  wrd_list=[]
  for word in words:
    if word.lower() not in stop_words:
      if word.isalpha():
        wrd_list.append(word)
  return wrd_list

In [22]:
#Applying on single row
list1=rem_punct_stopwords(df.reviewText[1].lower())
len(list1)

468

In [23]:
#Applying to entire column
df["Cleaned_reviewText"]=df.reviewText.apply(rem_punct_stopwords)
df["Cleaned_reviewText"]

0        [bought, first, served, faithfully, lost, trav...
1        [belated, review, feel, obliged, share, views,...
2        [hp, kicking, twenty, years, hp, years, old, s...
3        [started, finance, stuff, recently, went, look...
4        [simple, calculations, discounted, cash, flows...
                               ...                        
53253    [like, scale, power, three, different, ways, r...
53254    [accuteck, shippro, digital, scale, works, wel...
53255    [ship, lot, stuff, sell, small, parts, manuals...
53256    [great, little, scale, weigh, oz, lbs, cannot,...
53257    [asked, review, scale, almost, declined, mail,...
Name: Cleaned_reviewText, Length: 53258, dtype: object

In [0]:
#Finding unique words
def get_unique(wrd_list):
  wrd_array=np.array(wrd_list)
  uni_array=np.unique(wrd_array)
  return list(uni_array)


In [26]:
#Applying on single row
uni_wrdlist=get_unique(list1)
len(uni_wrdlist)

276

In [0]:
#Applying on entire column
unique_words=list(df.Cleaned_reviewText.apply(get_unique))

In [0]:
#Getting combinations of words
from itertools import combinations
def get_combo(uni_list):
  combo=combinations(uni_list, 2)
  return list(combo)


In [30]:
uni_combo=get_combo(uni_wrdlist)
len(uni_combo)
uni_combo

[('according', 'action'),
 ('according', 'alike'),
 ('according', 'almost'),
 ('according', 'always'),
 ('according', 'amazon'),
 ('according', 'amusing'),
 ('according', 'anniversary'),
 ('according', 'anyone'),
 ('according', 'ardent'),
 ('according', 'ask'),
 ('according', 'attempt'),
 ('according', 'based'),
 ('according', 'bear'),
 ('according', 'bearer'),
 ('according', 'belated'),
 ('according', 'benchmarking'),
 ('according', 'best'),
 ('according', 'better'),
 ('according', 'boy'),
 ('according', 'brandname'),
 ('according', 'broken'),
 ('according', 'built'),
 ('according', 'burying'),
 ('according', 'business'),
 ('according', 'calculator'),
 ('according', 'calculators'),
 ('according', 'case'),
 ('according', 'challenge'),
 ('according', 'charge'),
 ('according', 'com'),
 ('according', 'come'),
 ('according', 'companion'),
 ('according', 'computer'),
 ('according', 'computers'),
 ('according', 'continued'),
 ('according', 'continuing'),
 ('according', 'could'),
 ('according

In [0]:
 #Training the model
  model = gensim.models.Word2Vec(
          unique_words,
          size=150,
          window=10,
          min_count=2,
          iter=5)

In [0]:
#Getting similarity scores for each combination
def get_sim_scores(list3):

  for i in range(0,len(list3)):
    sim_scores=[]
    sim_scores.append(model.similarity(list3[i][0], list3[i][1]))
    print (list(map(lambda x, y:(x,y), list3, sim_scores)))

In [34]:
#Run on complete reviewText
for i in range(0, df.shape[0]):
  get_sim_scores(get_combo(get_unique(rem_punct_stopwords(df.reviewText[i]))))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `similarity` (Method will be removed in 4.0.0, use self.wv.similarity() instead).
  """
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[(('another', 'applications'), 0.7698415)]
[(('another', 'applications'), 0.8746815)]
[(('another', 'applications'), 0.7849233)]
[(('another', 'applications'), 0.73473126)]
[(('another', 'applications'), 0.9053788)]
[(('another', 'applications'), 0.796638)]
[(('another', 'applications'), 0.86065495)]
[(('another', 'applications'), 0.7255431)]
[(('another', 'applications'), 0.7356522)]
[(('another', 'applications'), 0.614744)]
[(('another', 'applications'), 0.8063459)]
[(('another', 'applications'), 0.53470206)]
[(('another', 'applications'), 0.68889666)]
[(('another', 'applications'), 0.19518042)]
[(('another', 'applications'), 0.51071876)]
[(('another', 'applications'), 0.436798)]
[(('another', 'applications'), 0.34711528)]
[(('another', 'applications'), 0.36839193)]
[(('another', 'applications'), 0.12004025)]
[(('another', 'applications'), 0.113314115)]
[(('another', 'applications'), 0.14274663)]
[(('another', 'applications'), 0.036781635)]
[(('another', 'applications'), 0.07900919)]

KeyError: ignored